In [1]:
import numpy as np
import seaborn as sns
import torch
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt 

In [2]:
# Hyperparameters
k = 160
alpha = 8e-4
l2_lambda = 0.01 # Regularization for the Item (Movie) Latent Component
l2_mu = 0.01 # Regularization for the User Latent Component
epochs = 1000
epochs_list = [225, 250, 275, 300, 325, 350, 375, 400]
momentum = 0.1
# patience = 10  # Number of epochs to wait for improvement before stopping
# tolerance = 1e-4  # Minimum improvement threshold

#blows up with 1e-1, 1e-2
alpha_list = [1e-3, 8e-4, 5e-4]
k_list = [90, 100, 110, 120, 130, 140, 150, 160]
mu_list = [0.01, 0.005, 0.001]
lambda_list = [0.01, 0.005, 0.001]
RMSE_list = []

In [3]:
# Ratings Matrix

# I'll stick to how the data was provided and use a different convention than
# the one used in class.
# Users will be on the rows of the matrix and items (in this case, movies) as
# columns.
R = np.load('ratings_train.npy')
R_test = np.load('ratings_test.npy')
# tensor = np.array(torch.load('tensor.pth'))
# users = []
# for user in R:
    
#     use = user[~np.isnan(user)]
#     user = [np.mean(use)] * len(tensor[0])
    
#     users.append(user)
# users = np.array(users)
# #print(np.shape(tensor))
# print(tensor)
# print(users)
#print(R_test)

[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
[[4.35211268 4.35211268 4.35211268 ... 4.35211268 4.35211268 4.35211268]
 [3.92857143 3.92857143 3.92857143 ... 3.92857143 3.92857143 3.92857143]
 [1.57692308 1.57692308 1.57692308 ... 1.57692308 1.57692308 1.57692308]
 ...
 [3.05576208 3.05576208 3.05576208 ... 3.05576208 3.05576208 3.05576208]
 [3.4375     3.4375     3.4375     ... 3.4375     3.4375     3.4375    ]
 [3.7414248  3.7414248  3.7414248  ... 3.7414248  3.7414248  3.7414248 ]]


/var/folders/h_/t0f_b8n152d3c4c7jj83pf0w0000gn/T/ipykernel_95524/1118610029.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tensor = np.array(torch.load('tensor.pth'))


In [4]:
# Dimensions of R Matrix
m, n = R.shape
#610 k 4980 k
#610 k 
print(m, n)

610 4980


In [5]:
# Set of User-Movie pairs with no ratings
S = np.isnan(R)
S_train_finite = np.isfinite(R)
S_test_finite = np.isfinite(R_test)

In [6]:
# Initializing User and Item Latent Component (U and I Matrices)
# To incorporate biases, it's easier to simply add to columns to both matrices,
# and the last column with 1s.
if k != 0:
    sd = 1/k

    U = np.random.normal(scale=sd, size=(m, k+2))
    I = np.random.normal(scale=sd, size=(n, k+2))
    I = tensor
    
    # Initialize momentum terms (velocities) for U and V
    v_U = np.zeros_like(U)
    v_I = np.zeros_like(I)

else:
    U = np.random.random(size=(m, k_dim))
    I = np.random.random(size=(n, k_dim))

U[:, -1] = 1
I[:, -2] = 1

In [7]:
def rmse(pred, target, mask):
    pred2 = pred[mask]
    target2 = target[mask]
    
    return np.sqrt(np.mean((pred2 - target2)**2))

def accuracy(pred, target):
    
    return np.sum(np.equal(pred, target)) / np.size(pred)

def round_to_nearest_half(matrix):
    return np.ceil(matrix * 2) / 2

def fit_distribution(predicted_R, R):
    # Assume U, V are the factor matrices from matrix factorization
   
    
    # Calculate mean and std of observed non-zero values in R
    R_values = R.flatten()
   
    mean_R, std_R = np.mean(R_values), np.std(R_values)
    
    # Calculate mean and std of predicted values in predicted_R
    predicted_R_values = predicted_R.flatten()
    mean_pred, std_pred = np.mean(predicted_R_values), np.std(predicted_R_values)
    
    # Scale and shift to align the predicted mean and std with the observed ones
    scaled_predicted_R = ((predicted_R - mean_pred) / std_pred) * std_R + mean_R

    # Apply further regularization if needed, e.g., by clipping or binned transformation
    scaled_predicted_R = np.clip(scaled_predicted_R, np.min(R_values), np.max(R_values))
    
    return scaled_predicted_R

In [8]:
# #hyperparameter training

# hyperparameter1 = alpha_list
# hyperparameter2 = k_list
# hyperparameter3 = epochs_list
# RMSE_list = []
# for ia in hyperparameter1:
    
#     for ib in hyperparameter2:
#         for ic in hyperparameter3:
#             # l2_mu = ia
#             # l2_lambda = ib
#             alpha = ia
#             k = ib
#             sd = 1/k
#             U = np.random.normal(scale=sd, size=(m, k+2))
#             I = np.random.normal(scale=sd, size=(n, k+2))
#             v_U = np.zeros_like(U)
#             v_I = np.zeros_like(I)
#             U[:, -1] = 1
#             I[:, -2] = 1
            
#             losses = []
#             # Initialize variables for early stopping
#             best_validation_loss = float('inf')
#             epochs_without_improvement = 0
#             epochs = ic
#             for _ in tqdm(range(epochs)):
#                 # Setting entries with no ratings to zero
#                 Rhat = np.dot(U, I.T)
#                 Rhat[S] = 0
            
#                 E = np.nan_to_num(R) - Rhat
            
#                 # Loss
#                 loss = np.sqrt(np.linalg.norm(E, 'fro'))
#                 losses.append(loss)
#                 #print(loss)
            
#                 # Gradients
#                 dU = -np.dot(E, I) + l2_mu*U
#                 dI = -np.dot(E.T, U) + l2_lambda*I
#                 # Update velocity for U and V with momentum
#                 v_U = (momentum * v_U) - (alpha * dU)
#                 v_I = (momentum * v_I) - (alpha * dI)
                
#                 # Update U and V using momentum
#                 U += v_U
#                 I += v_I
#                 # U -= alpha*dU
#                 # I -= alpha*dI
                
#                 # Reset items in last columns to 1s.
#                 U[:, -1] = 1
#                 I[:, -2] = 1
               
                
            
#                 # # Check if validation loss improved
#                 # if loss < best_validation_loss - tolerance:
#                 #     best_validation_loss = loss
#                 #     epochs_without_improvement = 0  # Reset patience counter
#                 # else:
#                 #     epochs_without_improvement += 1  # Increment patience counter
            
                
            
#                 # # Early stopping condition
#                 # if epochs_without_improvement >= patience:
#                 #     print(_)
#                 #     break
    
            
#             Rhat_train_final = np.where(S_train_finite, np.dot(U, I.T), 0)
#             Rhat_test_final = np.where(S_test_finite, np.dot(U, I.T), 0)
#             R_train_final = np.where(S_train_finite, np.nan_to_num(R), 0)
#             R_test_final = np.where(S_test_finite, np.nan_to_num(R_test), 0)
            
#             training_RMSE = rmse(Rhat_train_final, R_train_final, S_train_finite)
#             test_RMSE = rmse(Rhat_test_final, R_test_final, S_test_finite)
#             RMSE_list.append(test_RMSE)
#             test_RMSE_rounded = rmse(round_to_nearest_half(Rhat_test_final), R_test_final, S_test_finite)
#             RMSE_list.append(test_RMSE_rounded)
#             test_RMSE_fitted = rmse(fit_distribution(Rhat_test_final, R_test_final), R_test_final, S_test_finite)
#             RMSE_list.append(test_RMSE_fitted)
#             test_RMSE_fitted_rounded = rmse(round_to_nearest_half(fit_distribution(Rhat_test_final, R_test_final)), R_test_final, S_test_finite)
#             RMSE_list.append(test_RMSE_fitted_rounded)
#             test_RMSE_rounded_fitted = rmse(fit_distribution(round_to_nearest_half(Rhat_test_final), R_test_final), R_test_final, S_test_finite)
#             RMSE_list.append(test_RMSE_rounded_fitted)
#             print("k: " + str(ia) + ", alpha: " + str(ib))
#             print("training RMSE: " + str(training_RMSE))
#             print("test RMSE: " + str(test_RMSE))
#             print("test RMSE rounded: " + str(test_RMSE_rounded))
#             print("test RMSE fitted: " + str(test_RMSE_fitted))
#             print("test RMSE fitted rounded: " + str(test_RMSE_fitted_rounded))
#             print("test RMSE rounded fitted: " + str(test_RMSE_rounded_fitted))
#             print("loss: " + str(sum(losses)))
    

In [9]:

# print(RMSE_list)
# fig = sns.lineplot(RMSE_list)
# plt.xlabel("alpha")
# plt.ylabel("RMSE")


In [10]:
# fig = sns.lineplot(loss)

In [11]:
# #intialization using word and user embeddings
# U = users
# I = tensor
# print(U)
# print(I)

# empty_columns_1 = np.zeros((np.shape(U)[0], 2))
# empty_columns_2 = np.zeros((np.shape(I)[0], 2))
# U = np.hstack((U, empty_columns_1))
# I = np.hstack((I, empty_columns_2))
# print(np.shape(U))
# print(np.shape(I))
# v_U = np.zeros_like(U)
# v_I = np.zeros_like(I)
# U[:, -1] = 1
# I[:, -2] = 1
# print(U)
# print(I)
# losses = []
# # Initialize variables for early stopping
# best_validation_loss = float('inf')
# epochs_without_improvement = 0
# epochs = 1000
# for _ in tqdm(range(epochs)):
#     # Setting entries with no ratings to zero
#     Rhat = np.dot(U, I.T)
#     Rhat[S] = 0
#     E = np.nan_to_num(R) - Rhat

#     # Loss
#     loss = np.sqrt(np.linalg.norm(E, 'fro'))
#     losses.append(loss)
#     #print(loss)

#     # Gradients

#     dU = -np.dot(E, I) + l2_mu*U
#     dI = -np.dot(E.T, U) + l2_lambda*I
    
#     # Update velocity for U and V with momentum
#     v_U = (momentum * v_U) - (alpha * dU)
#     v_I = (momentum * v_I) - (alpha * dI)
#     # Update U and V using momentum
#     U += v_U
#     I += v_I
#     # U -= alpha*dU
#     # I -= alpha*dI
    
#     # Reset items in last columns to 1s.
#     U[:, -1] = 1
#     I[:, -2] = 1
#     # print(U)
#     # print(I)
    

#     # # Check if validation loss improved
#     # if loss < best_validation_loss - tolerance:
#     #     best_validation_loss = loss
#     #     epochs_without_improvement = 0  # Reset patience counter
#     # else:
#     #     epochs_without_improvement += 1  # Increment patience counter

    

#     # # Early stopping condition
#     # if epochs_without_improvement >= patience:
#     #     print(_)
#     #     break


# Rhat_train_final = np.where(S_train_finite, np.dot(U, I.T), 0)
# Rhat_test_final = np.where(S_test_finite, np.dot(U, I.T), 0)
# R_train_final = np.where(S_train_finite, np.nan_to_num(R), 0)
# R_test_final = np.where(S_test_finite, np.nan_to_num(R_test), 0)

# training_RMSE = rmse(Rhat_train_final, R_train_final, S_train_finite)
# test_RMSE = rmse(Rhat_test_final, R_test_final, S_test_finite)
# RMSE_list.append(test_RMSE)
# test_RMSE_rounded = rmse(round_to_nearest_half(Rhat_test_final), R_test_final, S_test_finite)
# RMSE_list.append(test_RMSE_rounded)
# test_RMSE_fitted = rmse(fit_distribution(Rhat_test_final, R_test_final), R_test_final, S_test_finite)
# RMSE_list.append(test_RMSE_fitted)
# test_RMSE_fitted_rounded = rmse(round_to_nearest_half(fit_distribution(Rhat_test_final, R_test_final)), R_test_final, S_test_finite)
# RMSE_list.append(test_RMSE_fitted_rounded)
# test_RMSE_rounded_fitted = rmse(fit_distribution(round_to_nearest_half(Rhat_test_final), R_test_final), R_test_final, S_test_finite)
# RMSE_list.append(test_RMSE_rounded_fitted)
# print("k: " + str(k) + ", alpha: " + str(alpha))
# print("training RMSE: " + str(training_RMSE))
# print("test RMSE: " + str(test_RMSE))
# print("test RMSE rounded: " + str(test_RMSE_rounded))
# print("test RMSE fitted: " + str(test_RMSE_fitted))
# print("test RMSE fitted rounded: " + str(test_RMSE_fitted_rounded))
# print("test RMSE rounded fitted: " + str(test_RMSE_rounded_fitted))
# print("loss: " + str(sum(losses)))



[[4.35211268 4.35211268 4.35211268 ... 4.35211268 4.35211268 4.35211268]
 [3.92857143 3.92857143 3.92857143 ... 3.92857143 3.92857143 3.92857143]
 [1.57692308 1.57692308 1.57692308 ... 1.57692308 1.57692308 1.57692308]
 ...
 [3.05576208 3.05576208 3.05576208 ... 3.05576208 3.05576208 3.05576208]
 [3.4375     3.4375     3.4375     ... 3.4375     3.4375     3.4375    ]
 [3.7414248  3.7414248  3.7414248  ... 3.7414248  3.7414248  3.7414248 ]]
[[0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 1. 0.]]
(610, 948)
(4980, 948)
[[4.35211268 4.35211268 4.35211268 ... 4.35211268 0.         1.        ]
 [3.92857143 3.92857143 3.92857143 ... 3.92857143 0.         1.        ]
 [1.57692308 1.57692308 1.57692308 ... 1.57692308 0.         1.        ]
 ...
 [3.05576208 3.05576208 3.05576208 ... 3.05576208 0.         1.        ]
 [3.4375     3.4375     3.4375     ... 3.4375     0.         1.        ]
 [3.741

  0%|          | 0/1000 [00:00<?, ?it/s]

k: 946, alpha: 8e-08
training RMSE: 0.8013773052529161
test RMSE: 0.9705623956757031
test RMSE rounded: 1.014211629515705
test RMSE fitted: 0.9621306977467662
test RMSE fitted rounded: 1.0047989162413933
test RMSE rounded fitted: 0.957549030634426
loss: 16074.718296107598


In [ ]:
epochs = 375
for _ in tqdm(range(epochs)):
    # Setting entries with no ratings to zero
    Rhat = np.dot(U, I.T)
    Rhat[S] = 0
    E = np.nan_to_num(R) - Rhat

    # Loss
    loss = np.sqrt(np.linalg.norm(E, 'fro'))
    losses.append(loss)
    #print(loss)

    # Gradients

    dU = -np.dot(E, I) + l2_mu*U
    dI = -np.dot(E.T, U) + l2_lambda*I
    
    # Update velocity for U and V with momentum
    v_U = (momentum * v_U) - (alpha * dU)
    v_I = (momentum * v_I) - (alpha * dI)
    # Update U and V using momentum
    U += v_U
    I += v_I
    # U -= alpha*dU
    # I -= alpha*dI
    
    # Reset items in last columns to 1s.
    U[:, -1] = 1
    I[:, -2] = 1
    # print(U)
    # print(I)
    

Rhat_train_final = np.where(S_train_finite, np.dot(U, I.T), 0)
Rhat_test_final = np.where(S_test_finite, np.dot(U, I.T), 0)
R_train_final = np.where(S_train_finite, np.nan_to_num(R), 0)
R_test_final = np.where(S_test_finite, np.nan_to_num(R_test), 0)

training_RMSE = rmse(Rhat_train_final, R_train_final, S_train_finite)
test_RMSE = rmse(Rhat_test_final, R_test_final, S_test_finite)
RMSE_list.append(test_RMSE)
test_RMSE_rounded = rmse(round_to_nearest_half(Rhat_test_final), R_test_final, S_test_finite)
RMSE_list.append(test_RMSE_rounded)
test_RMSE_fitted = rmse(fit_distribution(Rhat_test_final, R_test_final), R_test_final, S_test_finite)
RMSE_list.append(test_RMSE_fitted)
test_RMSE_fitted_rounded = rmse(round_to_nearest_half(fit_distribution(Rhat_test_final, R_test_final)), R_test_final, S_test_finite)
RMSE_list.append(test_RMSE_fitted_rounded)
test_RMSE_rounded_fitted = rmse(fit_distribution(round_to_nearest_half(Rhat_test_final), R_test_final), R_test_final, S_test_finite)
RMSE_list.append(test_RMSE_rounded_fitted)
print("k: " + str(k) + ", alpha: " + str(alpha))
print("training RMSE: " + str(training_RMSE))
print("test RMSE: " + str(test_RMSE))
print("test RMSE rounded: " + str(test_RMSE_rounded))
print("test RMSE fitted: " + str(test_RMSE_fitted))
print("test RMSE fitted rounded: " + str(test_RMSE_fitted_rounded))
print("test RMSE rounded fitted: " + str(test_RMSE_rounded_fitted))
print("loss: " + str(sum(losses)))